## 4.1.1 データ駆動

> ニューラルネットワークは，画像に含まれる重要な特徴量までも「機械」が学習するのです

これ，よく言われる表現だが，まあどういう意味での主張か理解できた．  

1. 例えば回帰のニューラルネットワークモデルを考える．  
2. 一般的には，出力層での活性化関数は恒等関数．  
すなわち，出力層のノード(アウトカムの値そのもの)は，最後の隠れ層ノードの線形結合になってる．  
3. ここで，最後の隠れ層ノードっていうのは，マクロな視点で見ると，説明変数を線形変換・活性化で何度も何度も変換して得られた特徴量である．  
データから交互作用をめっちゃ考慮した特徴量を作った感じ．  
4. こう考えると，DNNは「重要な特徴量＝最終隠れ層ノード」を学習で獲得して，それで線形回帰している，と捉えられる．